## Numeric Data

The API serves normalized and as-reported financial information in JSON.

Calcbench uses cookie based authentication, you will need to use a client that handles cookies, an example using Python's requests package is below.

In [ ]:
import requests
import json
cb_email = "andrew@calcbench.com" # put your calcbench username here
cb_password = "password" # put your calcbench password here
CALCBENCH_SESSION = requests.Session()
CALCBENCH_API_URL_BASE = "https://www.calcbench.com/api/{0}"
r = CALCBENCH_SESSION.post('https://www.calcbench.com/account/LogOnAjax', 
                  {'email' : cb_email, 
                   'password' : cb_password, 
                   'rememberMe' : 'true'}, 
                  verify=True)
assert r.text == 'true', 'login failed'

#### Get tickers by industry.

In [ ]:
pharma_SIC_code = 2834
url = CALCBENCH_API_URL_BASE.format("companies?siccodes={0}".format(pharma_SIC_code))
pharma_companies = CALCBENCH_SESSION.get(url).json()
pharma_tickers = [co['ticker'] for co in pharma_companies]

### Normalized Data

In [ ]:
metrics = ['revenue', 'netincome', 'assets', 'stockholdersequity']
payload = {"start_year" : 2010,
           'start_period' : 1,
           'end_year' : 2014,
           'end_period' : 4,
           'company_identifiers' : ['ibm', 'goog'],
           'metrics' : metrics,
           }
url = CALCBENCH_API_URL_BASE.format("/NormalizedValues")
data = CALCBENCH_SESSION.post(url,
                              data=json.dumps(payload), 
                              headers={'content-type' : 'application/json'}
                              ).json()

In [ ]:
data[0]

### As Reported Data

In [ ]:
ticker = "ibm"
statement_type = "income" # one of (income, balance, cash)
as_reported_path = 'asreported/?companyIdentifier={0}&statementType={1}&periodType=annual'.format(ticker, statement_type)
response = CALCBENCH_SESSION.get(CALCBENCH_API_URL_BASE.format(as_reported_path), verify=True)
as_reported_data = json.loads(response.text)

In [ ]:
as_reported_data